In [1]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
from utils.evaluator import Evaluator

from sklearn.metrics import mean_squared_error
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torchmetrics import MeanSquaredError
from torch.utils.data import Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, BertModel
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

# Descargar la lista de stopwords si no está ya descargada
nltk.download('stopwords')

# Cargar las stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))


print("¿GPU disponible?:", tf.config.list_physical_devices('GPU'))

/home/liingfeng/Desktop/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-21 22:18:36.941469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734815917.010343  231672 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734815917.028490  231672 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 22:18:37.173538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-criti

¿GPU disponible?: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/liingfeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = Loader.load_NLP()

/home/liingfeng/Desktop/repos/Cars_Price_Prediction/NLP/../utils/loader.py:153: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
/home/liingfeng/Desktop/repos/Cars_Price_Prediction/NLP/../utils/loader.py:153: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
/home/liingfeng/Desktop/repos/Cars_Price_Prediction/NLP/../utils/loader.py:165: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the ori

In [3]:
def custom_concat(row, cols):
    # Construir la descripción con lógica condicional basada en el valor de la celda
    parts = []
    for col_name in cols:  # Cambio para iterar solo sobre las columnas especificadas
        if col_name in row.index:  # Verificar que el nombre de la columna esté en el DataFrame
            value = row[col_name]
            if value == "no tiene" or not isinstance(value, str):
                parts.append(f"no tiene {col_name}")
            else:
                parts.append(str(value))  # Convertir a string para evitar problemas al unir
    # Unir todas las partes con espacios
    return ' '.join(parts)

# Aplicar la función al DataFrame
def filter_train_data(train):
    descriptions = [col for col in train.columns if "description" in col]
    train['full_description'] = train.apply(custom_concat, axis=1, args=(descriptions,))
    filtered_columns = ["price", "km", "fuelType", "full_description"]
    train = train[filtered_columns]
    train.dropna(inplace=True)
    return train

train = filter_train_data(train)

km_scaler = StandardScaler()
train["km"] = km_scaler.fit_transform(train["km"].to_numpy().reshape(-1, 1))

price_scaler = StandardScaler()
train["price"] = price_scaler.fit_transform(train["price"].to_numpy().reshape(-1, 1))

/tmp/ipykernel_231672/2940014903.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(inplace=True)


In [4]:
verb_size = 128
model_name = 'dccuchile/bert-base-spanish-wwm-cased'  # BETO


In [5]:
train_texts, val_texts, train_labels, val_labels, train_km, val_km = train_test_split(
    train["full_description"],
    train["price"],
    train["km"],
    test_size=0.2,
    random_state=42,
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [7]:
train_encodings = tokenizer(
    list(train_texts), truncation=True, padding=True, max_length=verb_size
)
val_encodings = tokenizer(
    list(val_texts), truncation=True, padding=True, max_length=verb_size
)


In [8]:
class RegressionDataset(Dataset):
    def __init__(self, encodings, labels, km_values):
        self.encodings = encodings
        self.labels = labels.astype(np.float32)  # Ensure labels are float
        self.km_values = torch.tensor(km_values, dtype=torch.float32)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        item['km'] = self.km_values[idx]
        return item

    def __len__(self):
        return len(self.labels)


In [9]:
train_dataset = RegressionDataset(train_encodings, train_labels, train_km.values)
val_dataset = RegressionDataset(val_encodings, val_labels, val_km.values)

class CustomRegressionModel(torch.nn.Module):
    def __init__(self, bert_model_name, km_dim=1):
        super(CustomRegressionModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.km_layer = torch.nn.Linear(km_dim, 16)  # Process km separately
        self.regressor = torch.nn.Linear(self.bert.config.hidden_size + 16, 1)  # Combine BERT and km outputs

    def forward(self, input_ids, attention_mask, km, labels=None):
        # BERT outputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_cls_output = outputs.pooler_output  # [CLS] token representation

        # Process km
        if len(km.shape) == 1:
            km = km.unsqueeze(1)  # Ensure km has two dimensions
        km_output = self.km_layer(km)

        # Concatenate and pass to regression head
        combined_output = torch.cat((bert_cls_output, km_output), dim=1)
        logits = self.regressor(combined_output)

        loss = None
        if labels is not None:
            loss_fn = torch.nn.MSELoss()
            loss = loss_fn(logits.view(-1), labels.view(-1))

        return {"loss": loss, "logits": logits, "labels": labels}
    
model = CustomRegressionModel(model_name)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
mse_metric = MeanSquaredError()


def compute_metrics(pred):
    # Unpack predictions and ensure it's a tensor
    predictions = torch.tensor(pred.predictions[0]).flatten()  # Access first element of the tuple and flatten
    labels = torch.tensor(pred.label_ids)  # Labels are directly accessible

    # Compute Mean Squared Error
    mse = mean_squared_error(labels, predictions)

    return {"mse": mse}


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=45,
    per_device_train_batch_size=84,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy='epoch',  # Evaluar al final de cada época
    save_strategy='epoch',  # Guardar modelo al final de cada época
    logging_strategy='steps',
    logging_steps=10,
    logging_dir='./logs',
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_mse',  # Usar eval_loss como métrica principal
    disable_tqdm=False
)


/home/liingfeng/Desktop/pytorch/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [22]:
trainer.train()
output_model_dir = './final_model'  
trainer.save_model(output_model_dir)
metrics = trainer.evaluate()
print("Returned metrics:", metrics)


                                                       
100%|█████████▉| 70630/70785 [9:10:58<01:06,  2.33it/s]

{'loss': 0.0343, 'grad_norm': 0.6809468269348145, 'learning_rate': 1.102653482250836e-07, 'epoch': 44.9}


                                                       
100%|█████████▉| 70640/70785 [9:11:03<01:02,  2.33it/s]

{'loss': 0.0323, 'grad_norm': 0.9654127359390259, 'learning_rate': 1.0315145479120724e-07, 'epoch': 44.91}


                                                       
100%|█████████▉| 70650/70785 [9:11:07<00:58,  2.31it/s]

...

{'eval_loss': 0.07916933298110962, 'eval_mse': 0.07916933298110962, 'eval_runtime': 58.3152, 'eval_samples_per_second': 566.37, 'eval_steps_per_second': 8.866, 'epoch': 45.0}


                                                       
100%|██████████| 70785/70785 [9:13:07<00:00,  2.13it/s]


{'train_runtime': 33187.6686, 'train_samples_per_second': 179.133, 'train_steps_per_second': 2.133, 'train_loss': 0.06284672509308646, 'epoch': 45.0}


100%|██████████| 517/517 [00:58<00:00,  8.82it/s]

Returned metrics: {'eval_loss': 0.11178498715162277, 'eval_mse': 0.11178497970104218, 'eval_runtime': 58.7162, 'eval_samples_per_second': 562.502, 'eval_steps_per_second': 8.805, 'epoch': 45.0}


In [23]:
# Realizar predicciones con el conjunto de validación
predictions = trainer.predict(val_dataset)

predicted_values = price_scaler.inverse_transform(predictions.predictions[0])  
true_values = price_scaler.inverse_transform(val_labels.values.reshape(-1, 1))  

100%|██████████| 517/517 [00:57<00:00,  8.98it/s]


In [24]:
print(predicted_values.shape)
print(true_values.shape)
Evaluator.eval_regression(y_pred=np.array(predicted_values.ravel()), y_true=np.array(true_values.ravel()), plot=False, n_features=2, regressor_name="BETO")
Evaluator.save("BETO")

(33028, 1)
(33028, 1)
MAE: 3556.3666443686384 
 MSE: 51862692.056626245 
 RMSE: 7201.5756648546185 
 R2: 0.8867678010017715 
 MAPE: 18.22119213068934
Mean: 3556.3666443686384
Variance: 39214948.34744841
Standard Deviation: 6262.183991823333
r2_adjusted: 0.8867609436392281
Regression results saved to: evaluation/BETO_regression.tex
Classification results saved to: evaluation/BETO_classification.tex
